In [2]:
#for multiple image input(should do a naming convention from 0.png,1.png,2.png,..)

import cv2
import os
import pickle
import numpy as np

def decaptcha(filenames, input_folder):
    filenames1 = []

    for i in range(len(filenames)):
        input_image_path = os.path.join(input_folder, f"{i}.png")

        I = cv2.imread(input_image_path, 0)

        crop = I[0:100, 358:458]

        _, I_binary = cv2.threshold(crop, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

        H = np.ones((3, 3), dtype=np.float32) / 9

        I_fil = I_binary

        SE = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 4))

        # Perform morphological closing
        Iclosed = cv2.morphologyEx(I_fil, cv2.MORPH_CLOSE, SE)

        # Subtract the closed image from the filtered image
        Isub = cv2.subtract(Iclosed, I_fil)

        # Invert the subtracted image
        I_new = cv2.bitwise_not(Isub)

        # Subtract the inverted image from the filtered image
        I_clear = cv2.bitwise_not((cv2.subtract(I_new, I_fil)))
        I_clear_resized = cv2.resize(I_clear, (100, 100))  # Adjust the size as needed

        image1 = np.array(I_clear_resized).flatten()

        filenames1.append(image1)

    pick = open('model.sav', 'rb')
    model = pickle.load(pick)
    pick.close()

    prediction = model.predict(filenames1)

    labels = []

    for pred in prediction:
        print
        if pred == 'EVEN':
            labels.append("EVEN")
        else:
            labels.append("ODD")

    return labels

input_folder = '/content/drive/MyDrive/Colab Notebooks/test_data'

filenames = os.listdir(input_folder)

decaptcha(filenames, input_folder)

['ODD', 'ODD', 'ODD', 'EVEN', 'EVEN', 'ODD', 'EVEN']

In [4]:
#for single image input

import cv2
import os
import pickle
import numpy as np

def decaptcha(input_image_path):
    I = cv2.imread(input_image_path, 0)

    crop = I[0:100, 358:458]

    _, I_binary = cv2.threshold(crop, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    H = np.ones((3, 3), dtype=np.float32) / 9

    I_fil = I_binary

    SE = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 4))

    # Perform morphological closing
    Iclosed = cv2.morphologyEx(I_fil, cv2.MORPH_CLOSE, SE)

    # Subtract the closed image from the filtered image
    Isub = cv2.subtract(Iclosed, I_fil)

    # Invert the subtracted image
    I_new = cv2.bitwise_not(Isub)

    # Subtract the inverted image from the filtered image
    I_clear = cv2.bitwise_not((cv2.subtract(I_new, I_fil)))
    I_clear_resized = cv2.resize(I_clear, (100, 100))  # Adjust the size as needed

    image1 = np.array(I_clear_resized).flatten()

    filenames1 = [image1]

    pick = open('model.sav', 'rb')
    model = pickle.load(pick)
    pick.close()

    prediction = model.predict(filenames1)

    if prediction[0] == 'EVEN':
        predicted_label = "EVEN"
    else:
        predicted_label = "ODD"

    return predicted_label

input_image_path = '/content/drive/MyDrive/Colab Notebooks/test_data/3.png'  # Replace with the actual image path
predicted_label = decaptcha(input_image_path)
print(f"Predicted Label: {predicted_label}")

Predicted Label: EVEN
